<a href="https://colab.research.google.com/github/kharaderushikesh/face-mask-detection-using-cnn/blob/main/face_mask_detection_using_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Kaggle API
!pip install -q kaggle

# Make Kaggle folder and copy your API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset from Kaggle
!kaggle datasets download -d omkargurav/face-mask-dataset

# Unzip the dataset
!unzip -q face-mask-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset
License(s): unknown
face-mask-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace data/with_mask/with_mask_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setup ImageDataGenerator with rescaling and validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training data generator
train_generator = datagen.flow_from_directory(
    '/content/data',              # Path to the main data directory
    target_size=(128, 128),       # Resize all images to 128x128
    batch_size=32,                # Number of images per batch
    class_mode='binary',          # For binary classification
    subset='training'             # This is the training subset
)

# Validation data generator
val_generator = datagen.flow_from_directory(
    '/content/data',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'           # This is the validation subset
)


In [ ]:
# Build the CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary output
])


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


In [ ]:
# Evaluate model
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy = {acc}")


In [ ]:
# Plot Accuracy and Loss
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()
plt.show()


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Upload an image or provide image path
image_path = input("Enter the image path: ")

# Read and preprocess
img = cv2.imread(image_path)
cv2_imshow(img)
resized_img = cv2.resize(img, (128, 128))
scaled_img = resized_img / 255.0
reshaped_img = np.reshape(scaled_img, (1, 128, 128, 3))

# Predict
pred = model.predict(reshaped_img)
print("Raw Prediction:", pred)

if pred[0][0] > 0.5:
    print("😷 The person is wearing a mask.")
else:


    print("❌ The person is NOT wearing a mask.")
